# Loads and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Download and Import Latent Space data and corresponding Original Data from S3

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/treated0_l.p ../../temp/ldata_knn/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/match0_l.p ../../temp/ldata_knn/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/treated0_o.p ../../temp/odata_knn/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/match0_o.p ../../temp/odata_knn/match0_o.p

In [4]:
import pickle
with open(path+'/ldata_knn_h/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_knn_h/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata_knn_h/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_knn_h/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [5]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [6]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

$$d = |\frac{(\bar{x_t}-\bar{x_c})}{\sqrt{\frac{s_t^2 + s_c^2}{2}}}| $$

In [7]:
def ASMD_cont_calculate(tf, cf):
    t_mean = np.mean(tf.values)
    c_mean = np.mean(cf.values)
    t_var = np.var(tf.values)
    c_var = np.var(cf.values)
    d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
    return d

In [8]:
def ASMD_binary_calculate(tf, cf):
    pt = np.mean(tf.values)
    pc = np.mean(cf.values)
    d = np.abs((pt-pc)/(pt*(1-pt)+pc*(1-pc)))
    return d
    

In [9]:
def ASMD_final(cols, col_cat, cdf, tdf):
    ASMD = pd.DataFrame()
    cf = cdf[cols]
    tf = tdf[cols]
    for i, (col_cat_each, col_name_each) in enumerate(zip(col_cat, cols)):
        if col_cat_each == 'c':
            d = ASMD_cont_calculate(tf[col_name_each], cf[col_name_each])
        else:
            d = ASMD_binary_calculate(tf[col_name_each], cf[col_name_each])
        ASMD.loc[0, col_name_each] = d 
    return ASMD
    

### Latent Space

In [10]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [11]:
from itertools import repeat
given_value ='c'
latent_cols_cat=[]
latent_cols_cat.extend(repeat(given_value,len(latent_cols)))

In [12]:
ASMD = ASMD_final(latent_cols, latent_cols_cat, controls, treated)
ASMD

x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15      x16
0  0.010518  0.013715  0.015939  0.015213  0.023011  0.013851  0.014753  0.003831  0.013168  0.016027  0.015802  0.012384  0.014556  0.012493  0.013036  0.00913

### Original Space with George's Features only

In [13]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']
george_cols_cat = ['c', 'b', 'c', 'b', 'b', 'b', 'b']

In [14]:
ASMD = ASMD_final(george_cols, george_cols_cat, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.016108  0.018548   0.022818    0.200697    0.051687         0.052554        0.026093

### Original Space with all the features

In [15]:
original_cols = controls_orig.columns.to_list()[2:]

In [16]:
l1 = ['c', 'b', 'b', 'c', 'c', 'c']

given_value ='b'

orig_CC_cols_cat=[]
orig_CC_cols_cat.extend(repeat(given_value,len(original_cols)-len(l1)))

original_cols_cat = l1 + orig_CC_cols_cat

In [17]:
ASMD = ASMD_final(original_cols, original_cols_cat, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [18]:
ASMD.T

0
AGE_AT_MIDMONTH             0.016108
MBR_GNDR                    0.018548
MEDICAID_LOB                0.611997
ALLOW_AMT                   0.022818
ALLOW_ER                    0.028046
ALLOW_IP                    0.027794
CC_ADHD                     0.060364
CC_ALZHEIMER                1.001018
CC_ALZHEIMER_PLUS           0.768182
CC_AMI                      0.508840
CC_ANEMIA                   0.026045
CC_ANXIETY_DISORDER         0.018692
CC_ARTHRITIS                0.004616
CC_ASTHMA                   0.006172
CC_ATRIAL_FIB               0.152132
CC_AUTISM                   1.001567
CC_BIPOLAR                  0.043391
CC_BPH                      0.169877
CC_BREAST_CANCER            0.221863
CC_CATARACT                 0.049459
CC_CEREBRAL_PALSY           0.141520
CC_CHRONIC_KIDNEY           0.063561
CC_COLORECTAL_CANCER        0.269726
CC_COPD                     0.086796
CC_CYSTIC_FIBROSIS          0.441930
CC_DEPRESSION               0.019106
CC_DEPRESSIVE_DISORDERS     0.198428
CC_DEVELOP_DELAYS           0.180809
CC_DIABETES                 0.043962
CC_DISAB_DX_CNT             0.019579
CC_DX_CNT                   0.002455
CC_ENDOMETRIAL_CANCER       0.026326
CC_EPILEPSY                 0.196746
CC_FIBROMYALGIA             0.074950
CC_GLAUCOMA                 0.022238
CC_HEARING_IMPAIR           0.027153
CC_HEART_FAILURE            0.005400
CC_HEPATITIS_A              1.000180
CC_HEPATITIS_B_ACUTE        0.201156
CC_HEPATITIS_B_CHRONIC      0.436796
CC_HEPATITIS_C_ACUTE             NaN
CC_HEPATITIS_C_CHRONIC      0.461284
CC_HEPATITIS_C_UNSPECIFIED  0.276532
CC_HEPATITIS_D                   NaN
CC_HEPATITIS_E                   NaN
CC_HEPATITIS_GEN            0.334386
CC_HIP_FRACTURE             1.000325
CC_HYPERLIPIDEMIA           0.045748
CC_HYPERTENSION             0.052554
CC_HYPOTHYROID              0.026093
CC_INT_DISAB                0.414205
CC_ISCHEMIC                 0.014166
CC_LEARN_DISAB              0.237478
CC_LEUKEMIAS                0.328501
CC_LIVER                    0.019012
CC_LUNG_CANCER              0.357704
CC_MIGRAINE                 0.087867
CC_MOBILITY_IMPAIR          0.116304
CC_MULTIPLE_SCLEROSIS       0.226877
CC_MUSCULAR_DYSTROPHY       1.000667
CC_OBESITY                  0.051687
CC_OSTEOPOROSIS             0.172118
CC_PERIPHERAL               0.143048
CC_PERSONALITY              0.052132
CC_PRESSURE                 0.031200
CC_PROSTATE_CANCER          0.111797
CC_PTSD                     0.050639
CC_SCHIZOPHRENIA            0.408038
CC_SCHIZOPHRENIA_OTHER      0.447332
CC_SPINAL_CORD              0.465869
CC_SPINA_BIFIDA             0.498280
CC_STROKE                   0.272433
CC_TOBACCO                  0.200697
CC_TRAUMATIC_BRAIN          0.060061
CC_VISUAL_IMPAIR            0.238406

In [19]:
(ASMD.loc[0,:]>0.1).sum()

38